In [1]:
import pandas as pd
import numpy as np
dt = pd.read_pickle('Loan_binning.pickle')
dt

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot,JOININGYEAR_BIN,AGE_BIN,EXPERIENCEINCURRENTDOMAIN_BIN
1,Bachelors,2013,C,1,28,Female,No,3,1,"(2011.999, 2013.0]","(27.0, 28.0]","(2.0, 3.0]"
2,Bachelors,2014,B,3,38,Female,No,2,0,"(2013.0, 2014.0]","(34.0, 38.0]","(1.0, 2.0]"
3,Masters,2016,A,3,27,Male,No,5,1,"(2015.0, 2016.0]","(26.0, 27.0]","(4.0, 5.0]"
4,Masters,2017,C,3,24,Male,Yes,2,1,"(2016.0, 2017.0]","(21.999, 24.0]","(1.0, 2.0]"
6,Bachelors,2015,B,3,38,Male,No,0,0,"(2014.0, 2015.0]","(34.0, 38.0]","(-0.001, 1.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...
4647,Bachelors,2016,C,3,30,Male,No,2,0,"(2015.0, 2016.0]","(29.0, 31.0]","(1.0, 2.0]"
4648,Bachelors,2013,A,3,26,Female,No,4,0,"(2011.999, 2013.0]","(24.0, 26.0]","(3.0, 4.0]"
4649,Masters,2013,C,2,37,Male,No,2,1,"(2011.999, 2013.0]","(34.0, 38.0]","(1.0, 2.0]"
4650,Masters,2018,B,3,27,Male,No,5,1,"(2017.0, 2018.0]","(26.0, 27.0]","(4.0, 5.0]"


In [5]:
def num_event(event, dataframe ,field, val):
    if event == 'good':
        event = 1
    else:
        event = 0
    cond_main = dataframe[field] == val
    cond_event = dataframe['LeaveOrNot'] == event
    n = len(dataframe[cond_main & cond_event].index)
    return n


def dist_event(event, dataframe ,field, val):
    n = num_event(event, dataframe, field, val)
    sumd = 0
    all_elements = set(dataframe[field].to_numpy())
    for a in all_elements:
        s = num_event(event, dataframe, field, a)
        sumd = sumd + s
    return n/sumd

def WOE(dataframe, field, val):
    dg = dist_event('good', dataframe, field, val)
    db = dist_event('bad', dataframe, field, val)
    w = np.log(dg / db)
    return w

def IV(dataframe, field):
    elements = set(dataframe[field].to_numpy())
    n = len(elements)
    sumiv = 0
    for i in elements:
        dg = dist_event('good', dataframe, field, i)
        db = dist_event('bad', dataframe, field, i)
        s = (dg-db) * WOE(dataframe, field, i)
        sumiv = sumiv + s
    return sumiv   

fields = ['Education', 'PaymentTier', 'Age', 'Gender', 'EverBenched','JOININGYEAR_BIN','EXPERIENCEINCURRENTDOMAIN_BIN']
list_woe=[]
list_attr=[]
list_val=[]
list_ngood = []
list_nbad = []
list_dgood = []
list_dbad = []

for f in fields:
    the_set = set(dt[f].to_numpy())
   
    for s in the_set:
        n_good = num_event('good', dt, f, s)
        n_bad = num_event('bad', dt, f, s)
        d_good = dist_event('good', dt, f, s)
        d_bad = dist_event('bad', dt, f, s)
        woe = WOE(dt, f, s)

        if 'BIN' in f:
            s = str(s)
        list_attr.append(f)
        list_val.append(s)
        list_ngood.append(n_good)
        list_nbad.append(n_bad)
        list_dgood.append(d_good)
        list_dbad.append(d_bad)
        list_woe.append(woe)



x = list(zip(list_attr, list_val, list_ngood, list_nbad, list_dgood, list_dbad,list_woe)) 
xlabel = ['ATTR', 'VAL', 'N_GOOD', 'N_BAD', 'DIST_GOOD', 'DIST_BAD','WOE']
df = pd.DataFrame(x, columns = xlabel )

df = df.sort_values(by=['ATTR', 'VAL'])

df = df.reset_index(drop=True)



df

,ATTR,VAL,N_GOOD,N_BAD,DIST_GOOD,DIST_BAD,WOE
0,Age,22,10,18,0.009001,0.008384,0.071024
1,Age,23,10,19,0.009001,0.008850,0.016957
2,Age,24,100,172,0.090009,0.080112,0.116487
3,Age,25,114,164,0.102610,0.076386,0.295143
4,Age,26,164,280,0.147615,0.130415,0.123888
5,Age,27,155,287,0.139514,0.133675,0.042754
6,Age,28,134,321,0.120612,0.149511,-0.214790
7,Age,29,58,106,0.052205,0.049371,0.055815
8,Age,30,46,102,0.041404,0.047508,-0.137520
9,Age,31,27,68,0.024302,0.031672,-0.264860


In [8]:
df.to_pickle('attr.pickle')

In [7]:
df_iv = pd.DataFrame({'ATTR': [], 'IV': []})
for f in fields:
    d = pd.DataFrame({'ATTR':[f], 'IV': [IV(dt, f)]})
    df_iv = df_iv.append(d, ignore_index = True)
df_iv

,ATTR,IV
0,Education,0.105312
1,PaymentTier,0.300393
2,Age,0.035247
3,Gender,0.203643
4,EverBenched,0.029166
5,JOININGYEAR_BIN,1.162231
6,EXPERIENCEINCURRENTDOMAIN_BIN,0.007080
